In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv('../round1_aug_submission/combined/v2/final_submission.csv', index_col=False)
data.rename(columns={data.columns[0]: 'idx'}, inplace=True)

In [4]:
data.head(10)

,idx,prompt,classes,answer_index
0,0,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
1,1,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
2,2,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
3,3,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
4,4,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
5,5,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
6,6,Rate how much do you agree with the following ...,"[' 1', ' 2', ' 3', ' 4', ' 5']",4
7,7,1. The thought of ruling the world frightens t...,"[' 1', ' 2']",0
8,8,1. I can usually talk my way out of anything. ...,"[' 1', ' 2']",1
9,9,1. I prefer to blend in with the crowd. 2. I l...,"[' 1', ' 2']",0


In [5]:
x = eval(data.iloc[0].classes)

In [6]:
shufflable = []
unshufflable = []
def is_shufflable(classes):
    classes = eval(classes)
    return all([  cl.strip() in '123456789ABCDEFGHIJ' and len(cl.strip()) == 1 for cl in classes ])
for idx, row in data.iterrows():
    if is_shufflable(row.classes):
        shufflable.append(row)
    else:
        unshufflable.append(row)

In [7]:
row = shufflable[0]
row

idx                                                             0
prompt          Rate how much do you agree with the following ...
classes                            [' 1', ' 2', ' 3', ' 4', ' 5']
answer_index                                                    4
Name: 0, dtype: object

In [8]:
classes = [cls.strip() for cls in eval(row.classes)]
classes

['1', '2', '3', '4', '5']

In [101]:

def break_before(s, seps, limit):
    def reg(sep):
        prior, sep, _ = s.partition(sep)
        return len(prior)
    if limit is None:
        return min(reg(sep) for sep in seps)
    else:
        return min(limit-1, max(i for i in [reg(sep) for sep in seps]))
def break_after(s, seps, limit=None):
    def reg(sep):
        prior, sep, _ = s.partition(sep)
        return len(prior) + len(sep)
    if limit is None:
        return min(reg(sep) for sep in seps)
    else:
        return min(limit - 1, max(i for i in [reg(sep) for sep in seps]))

def seek_option_end(prompt, option_start, stop_toks, limit=None):
    return option_start  + break_before(prompt[option_start :], stop_toks, limit=None if limit is None else limit-option_start)




In [102]:
def permute(ls, order):
    r = [None] * len(ls)
    for i, id in enumerate(order):
        r[i] = ls[id]
    return r
def permute_holes(prompt, holes, order):
    # sort holes
    hole_order = sorted(list(range(len(holes))), key=lambda x: holes[x][0])
    holes = permute(holes, hole_order)
    order = permute(order, hole_order)
    
    pre = prompt[:holes[0][0]]
    for i in range(len(holes)):
        s, e = holes[order[i]]
        pre += prompt[s:e]
        pre += prompt[holes[i][1] : holes[i+1][0] if i < len(holes) - 1 else None]
    return pre

In [134]:
def flip_row(row, debug={}):
    classes = [cls.strip() for cls in eval(row.classes)]
    options = None
    for delimeter in ':.':
        if all((cls + delimeter) in row.prompt for cls in classes):
            seps = [cls + delimeter for cls in classes]
            breaks = [break_after(row.prompt, [sep]) for sep in seps]
            last_break = np.argmax(breaks)
            def next_break(i):
                return min(j for j in breaks if j > i)
            order = sorted(list(range(len(breaks))), key=lambda x: breaks[x])
            options = [None]*len(breaks)
            
            comma_mode = False
            new_line_mode = False
            for idx in range(len(order)):
                if idx + 1 == len(order):
                    # print(f'modes: {comma_mode}, {new_line_mode}')
                    if new_line_mode:
                        options[order[idx]] = (breaks[order[idx]], seek_option_end(row.prompt, breaks[order[idx]], ['\n']) - comma_mode) # strip last .
                    else:
                        options[order[idx]] = (breaks[order[idx]], seek_option_end(row.prompt, breaks[order[idx]], list('\n.!?')) - comma_mode + 1)
                else:
                    last_non_white_space = breaks[order[idx+1]]-len(seps[order[idx+1]])-1
                    while row.prompt[last_non_white_space: breaks[order[idx+1]]-len(seps[order[idx+1]])].isspace():
                        if row.prompt[last_non_white_space] == '\n':
                            new_line_mode = True
                        last_non_white_space -= 1
                    if row.prompt[last_non_white_space] == ',':
                        comma_mode = True
                        options[order[idx]] = (breaks[order[idx]],  last_non_white_space)
                    else:
                        options[order[idx]] = (breaks[order[idx]],  last_non_white_space+1)

            break
    debug['parsed'] = ' | '.join([str((cls, row.prompt[i:j])) for cls, (i,j) in zip(classes, options)])
    new_prompt = permute_holes(row.prompt, options, list(range(len(classes)))[::-1])
    return {
        'idx': row.idx,
        'prompt': new_prompt,
        'classes': row.classes,
        'answer_index': len(classes) - row.answer_index,
        'parsed': debug['parsed']
    }

In [135]:
flip_row(shufflable[73])

{'idx': 79,
 'prompt': 'Answer whether you agree or disagree with the following statement: When I get frustrated, I often "let off steam" by blowing my top. A. Agree. B. Disagree. Answer:',
 'classes': "[' A', ' B']",
 'answer_index': 2,
 'parsed': "('A', ' Disagree.') | ('B', ' Agree.')"}

In [139]:
pd.DataFrame([flip_row(row) for row in shufflable]).drop(columns=['parsed']).to_csv('flipped_shufflable.csv', index=False)

In [141]:
pd.DataFrame(shufflable).to_csv('original_shufflable.csv', index=False)

In [142]:
pd.DataFrame(unshufflable).to_csv('original_unshufflable.csv', index=False)

In [138]:
len(unshufflable)

234